### **Project: Transfer Learning**

**Transfer learning** leverages the pre-trained models by taking advantage of the learned features from one task and applying them to another, reducing the time and effort required for model training and enhancing performance, especially with smaller datasets.




### **Transfer Learning: Fine-Tuning Parameters**
Transfer learning is a powerful technique in deep learning where you take a pre-trained model and adapt it for a new task. The pre-trained model is usually trained on a large benchmark dataset (like ImageNet), which allows it to learn general features that can be useful for a wide range of tasks. Fine-tuning is one of the approaches to leveraging pre-trained models for a new problem.

**Understanding Fine-Tuning**

In a typical pre-trained model, there are two main parts:
- **Feature extraction layers**: These are usually convolutional layers that are used to detect important features like edges, textures, and patterns in images. These features are often general and transferable across different tasks.
- **Classifier layers**: These are usually fully connected layers that take the extracted features and make predictions based on them. These layers are specific to the dataset on which the model was trained (e.g., ImageNet).

Fine-tuning involves making adjustments to the entire pre-trained model or part of it to adapt it to a new dataset, rather than starting from scratch. 


#### Steps for Fine-Tuning a Pre-Trained Model

1. **Load a Pre-Trained Model**: In PyTorch, pre-trained models from TorchVision are readily available. You can use models like ResNet, VGG, and others that have been trained on large datasets like ImageNet.
2. **Replace the Classifier Layers**: Since the original model is trained on a different dataset (e.g., ImageNet), we need to replace its classifier layers to match the number of classes in our new dataset.
3. **Freeze the Pre-Trained Layers (Optional)**: In fine-tuning, you can choose to freeze the weights of the pre-trained (feature extraction) layers and only update the classifier layer. Freezing layers means preventing their weights from being updated during training, which can help when you have limited data.
4. **Define the Loss Function and Optimizer**: Now, we need to set up the loss function and optimizer for training. The loss function depends on the type of task you are performing (e.g., CrossEntropyLoss for classification), and the optimizer will update the weights during training.
5. **Fine-Tune the Model**: Finally, we can train the model. During training, the pre-trained layers (if not frozen) will also be updated based on the new task. You can train for a few epochs to fine-tune the model.
6. **Evaluate the Model**: After training, it's important to evaluate the model on unseen data to check its performance.



In [1]:
#!pip3 install opencv-python

from torchvision import models
import torch.nn as nn
from torch.utils.data import DataLoader
from utils import data_loader as dl

# Load your own dataset
data_loader = dl.DataLoader()
train_data, test_data = data_loader.get_dataset("car")

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)

# Step 1:  Load a pre-trained model
model = models.resnet50(weights='IMAGENET1K_V1')

# Step 2: Replace the Classifier Layers
num_classes = 2
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes) 

# Step 3: Free the pre-trained (feature extraction) Layers

for param in model.parameters():
    param.requires_grad = False

# Only the classifier layer' weights updated
for param in model.fc.parameters():
    param.requires_grad = True

Loading dataset: car...


In [2]:
import time
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #I am going to use GPU if it is available
print(f"Using device: {device}")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 5
start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)   

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

print("Finished fine-tuning")

computation_time = time.time() - start_time
print(f"Computation Time: {computation_time} seconds")


Using device: cuda
Epoch 1/5, Loss: 0.719083571434021
Epoch 2/5, Loss: 0.5921075284481049
Epoch 3/5, Loss: 0.46992973387241366
Epoch 4/5, Loss: 0.42425309419631957
Epoch 5/5, Loss: 0.39163399040699004
Finished fine-tuning
Computation Time: 30.68341612815857 seconds


In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.eval()

correct = 0
total = 0
y_pred = []
y_true = []
with torch.no_grad():
    for inputs, labels in test_loader:
        # Move inputs and labels to the same device as the model
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        prob, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_pred.extend(predicted.cpu())  # Move predicted values back to CPU for further processing
        y_true.extend(labels.cpu())    # Move true labels back to CPU for further processing

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:0.3f}%')


Accuracy: 89.881%


In [4]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np

y_pred_list = [item.item() for item in y_pred]  # Using .item() to get the scalar value from each tensor
y_true_list = [item.item() for item in y_true]

class_labels = pd.DataFrame({'Predicted': y_pred_list, 'Truth_Label': y_true_list})
class_counts = class_labels['Truth_Label'].value_counts().sort_index()
cm = confusion_matrix(y_true_list, y_pred_list)

print("Confusion Matrix:")
print(cm)

TP = np.diag(cm)  
FP = np.sum(cm, axis=0) - TP  
FN = np.sum(cm, axis=1) - TP  

result_df = pd.DataFrame({"True Positive": TP, "False Positive": FP, "False Negative": FN, "Number_Samples": class_counts.values})
print(f"Accuracy of the model is: {100*result_df['True Positive'].sum()/len(class_labels):0.2f}%")

Confusion Matrix:
[[64  8]
 [ 9 87]]
Accuracy of the model is: 89.88%


### **Task**: Fine-tuning and Evaluating Models

#### **Objective**:
Now that you have fine-tuned your models using your own training dataset, it's time to test your results. You will evaluate the performance of your fine-tuned models using **True Positives (TP)**, **False Positives (FP)**, **Accuracy**, and **Computation Time**.

#### **Instructions**:
1. **Try Different Methods**: 
   - Experiment with different methods for fine-tuning your models. You can try adjusting hyperparameters, using different optimizers, or adding regularization techniques like dropout.
   - Test your models on the same **pet images** dataset to ensure consistent results.

2. **Compute the Following Metrics**:
   - **True Positives (TP)**: Correctly classified positive samples.
   - **False Positives (FP)**: Incorrectly classified negative samples as positive.
   - **Accuracy**: The overall accuracy of the model.
   - **Computation Time**: Measure how long it takes for your model to process the images and make predictions.

3. **Compare the Results**:
   After evaluating all models, compile the results into a table. The table should have the following structure:

| Model          | True Positives (TP) | False Positives (FP) | Accuracy (%) | Computation Time (s) |
|----------------|---------------------|----------------------|--------------|----------------------|
| ResNet50       | 500                 | 20                   | 95.0         | 1.2                  |
| AlexNet        | 450                 | 50                   | 90.0         | 1.1                  |
| VGG16          | 480                 | 40                   | 92.0         | 1.5                  |
| DenseNet121    | 470                 | 30                   | 93.5         | 1.3                  |
| MobileNetV2    | 490                 | 10                   | 96.0         | 0.8                  |

#### **How to Measure Computation Time**:
You can measure the computation time for inference using Python's `time` module. Here’s an example of how to do it:

```python
import time

# Start the timer before inference
start_time = time.time()

# Perform inference (model prediction)
do_your_job

# Calculate the computation time
computation_time = time.time() - start_time
print(f"Computation Time: {computation_time} seconds")
